In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
from collections import deque

In [2]:
base_price=[10,10,10]
target_trade_spend = 10515.58
intercept = 15
coef_1 =-3
coef_2 =0.004
target_edlp_spend = 1906.41
target_tpr_spend = 8609.172
# init = [1,1,0]
idx = range(3)
# week=3
# lb = [9.5,9.5,9.5,5,5,5,0,0,0]
# ub = [10,10,10,50,50,50,1,1,1]



In [3]:
def cal_sales_price(edlp, tpr, flag):
    base_price=[10,10,10]
    
    unit_sales = list(exp((log( edlp[i])*coef_1)* flag[i]
                 +(log(base_price[i])*coef_1+ tpr[i]*coef_2)*(1- flag[i])
                 +intercept) for i in  idx)
    
    price = list((base_price[i]*(1-tpr[i]/100)*(1-flag[i])) 
        +(edlp[i]*flag[i]) for i in idx)
    return unit_sales, price
    
def calc_dollar_sales(edlp, tpr, flag):
    
    base_price=[10,10,10]
    unit_sales, price = cal_sales_price(edlp, tpr, flag)
    
    dollar_sales = [unit_sales[i]*price[i] for i in idx]
    
    return dollar_sales
      
def calc_total_trade_spent(edlp, tpr, flag):
    
    base_price=[10,10,10]
    unit_sales, price = cal_sales_price(edlp, tpr, flag)
    
    trade_spent = [base_price[i]-price[i] for i in idx]
    total_trade_spent = [trade_spent[i]*unit_sales[i] for i in idx]
    
    return total_trade_spent
    
    
def calc_edlp_trade_spent(edlp, tpr, flag):
    
    base_price=[10,10,10]
    unit_sales, price = cal_sales_price(edlp, tpr, flag)
    
    trade_spent = [base_price[i]-price[i] for i in idx]
    total_trade_spent = [trade_spent[i]*unit_sales[i] for i in idx]
    
    edlp_trade_spent = [total_trade_spent[i]*flag[i] for i in idx]
    
    return edlp_trade_spent
    
def calc_tpr_trade_spent(edlp, tpr, flag):
    base_price=[10,10,10]
    unit_sales, price = cal_sales_price(edlp, tpr, flag)
    
    trade_spent = [base_price[i]-price[i] for i in idx]
    total_trade_spent = [trade_spent[i]*unit_sales[i] for i in idx]
    tpr_trade_spent = [total_trade_spent[i]*(1-flag[i]) for i in idx]
    
    return tpr_trade_spent
    
def calc_flag_util(flag):
    flag_util = [flag[i]*(1-flag[i]) for i in idx]
    return flag_util



In [4]:
def create_model(init):
    print("Init value= ",init)
    def initial(model,i):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    model.edlp = Var(idx, initialize=10, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr = Var(idx, initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag = Var(idx,initialize=initial,domain=Binary)
    model.obj = Objective(expr=sum(calc_dollar_sales(model.edlp, model.tpr, model.flag)) ,sense= maximize)
    model.c1 = Constraint(expr = sum(calc_total_trade_spent(model.edlp, model.tpr, model.flag)) == target_trade_spend )
    model.c3 = Constraint(expr = sum(calc_tpr_trade_spent(model.edlp, model.tpr, model.flag)) == target_tpr_spend )
    # model.c2 = Constraint(expr = sum(calc_edlp_trade_spent(model.variables)) == target_edlp_spend )
    model.c4 = Constraint(expr = sum(calc_flag_util(model.flag))==0 )
    return model

In [5]:
def get_info_from_results(results, info_string):
    i = str(results).lower().find(info_string.lower()) + len(info_string)
    value = ''
    while str(results)[i] != '\n':
        value = value + str(results)[i]
        i += 1
    return value

In [8]:
def call_solver(init, name='bonmin'):
    assert type(init) == list
    model = create_model(init)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    model.display()
#     print(results_1)
#     print(get_info_from_results(results_1, 'Time: '))

combinations=[[0,0,0],
              [0,0,1],
              [0,1,1],
#               [1,0,0],[1,0,1],[1,1,0],[0,1,0],
              [1,1,1]]    
for c in combinations:
    call_solver(c, 'baron')

Init value=  [0, 0, 0]
Model Spend_Optim

  Variables:
    edlp : Size=3, Index=edlp_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.731363542100528 :    10 : False : False : NonNegativeReals
          1 :   9.5 : 9.731220977762659 :    10 : False : False : NonNegativeReals
          2 :   9.5 :              10.0 :    10 : False : False : NonNegativeReals
    tpr : Size=3, Index=tpr_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :     5 : 5.000000000000022 :    50 : False : False : NonNegativeReals
          1 :     5 : 49.99999999999999 :    50 : False : False : NonNegativeReals
          2 :     5 : 23.93154835462502 :    50 : False : False : NonNegativeReals
    flag : Size=3, Index=flag_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 